# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
import pandas as pd

# Leer y limpiar los archivos
urls = [
    'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv',
    'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv',
    'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'
]
dfs = [pd.read_csv(url) for url in urls]

def clean_columns(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

file1, file2, file3 = map(clean_columns, dfs)

# --- Paso clave: eliminar duplicados por cliente antes de merge ---
file1 = file1.drop_duplicates(subset='customer')
file2 = file2.drop_duplicates(subset='customer')
file3 = file3.drop_duplicates(subset='customer')

for i, f in enumerate([file1, file2, file3], start=1):
    print(f"Archivo {i}: {f['customer'].nunique()} clientes únicos de {len(f)} registros")

# --- Usar merge con outer join ---
data = file1.merge(file2, on='customer', how='outer').merge(file3, on='customer', how='outer')

# Si hay columnas duplicadas, nos quedamos con una sola versión (de file1)
data = data.loc[:, ~data.columns.duplicated()]

# Eliminar duplicados
data = data.drop_duplicates()

# Eliminar filas con todos los valores nulos
data = data.dropna(how='all')

# Limpiar valores numéricos con $, comas y % y convertir a float
data['customer_lifetime_value'] = (
    data['customer_lifetime_value']
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)   # ← raw string para evitar warning
    .str.replace('%', '', regex=False)
    .astype(float)
)

# Pasar a minúsculas algunas columnas categóricas
categorical_cols = data.select_dtypes(include='object').columns
data[categorical_cols] = data[categorical_cols].apply(lambda x: x.str.lower().str.strip())



print(f"Total registros combinados: {len(data)}")

In [ ]:
print(data.info())
print(data.head())


In [ ]:
data.to_csv('clean_combined_data.csv', index=False)


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [ ]:
# Cargar dataset limpio
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')

# Calcular Revenue según fórmula
df['revenue'] = (df['monthly_premium_auto'] * df['months_since_policy_inception']) - df['total_claim_amount']

# Crear tabla resumen con pivot
pivot_revenue = (
    pd.pivot_table(
        df,
        values='revenue',
        index='sales_channel',
        aggfunc='sum'
    )
    .round(2)
    .sort_values(by='revenue', ascending=False)
)

print(pivot_revenue)


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here

# Convert date column to datetime
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# Extract month name
df['month'] = df['effective_to_date'].dt.month_name()

# Create summary table
complaints_summary = (
    df.groupby(['policy_type', 'month'])['number_of_open_complaints']
      .count()
      .reset_index()
      .rename(columns={'number_of_open_complaints': 'num_complaints'})
)

print(complaints_summary.head())
